In [1]:
import pandas as pd
from twarc import Twarc
import datetime
import time

In [2]:

columns = ["screen_name", "tweet_id","created_at","embeded_url","expanded_url","author","title",
           "content", "day_0_time","day_0_retweet_count"]
columns

['screen_name',
 'tweet_id',
 'created_at',
 'embeded_url',
 'expanded_url',
 'author',
 'title',
 'content',
 'day_0_time',
 'day_0_retweet_count']

## Should we add favorite_count as well?

In [3]:
df = pd.DataFrame(columns=columns)

def export_dataset(df):
    name = "../data/dataset_{}.csv".format(datetime.datetime.now().strftime("%B_%d_%y_%H_%M_%S"))
    df.to_csv(name)

# export_dataset(df)
df.head()

,screen_name,tweet_id,created_at,embeded_url,expanded_url,author,title,content,day_0_time,day_0_retweet_count


In [4]:
consumer_key='58wlGpzEOwum6nghvA8tj6nz6'
consumer_secret='b04KqyzRyquIiYGwqv9B12hRqjEj9tvOp53DtoFZJtetaZ7cZE'
access_token='1243275961824612358-65qxIlXTEhjw0jlbBaK4I0olH9nRP9'
access_token_secret='wYdkYaNCakb7sX5PnHjF1u0oJc7Y73iijqbJ4VeB4yyEd'
 
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

a tweet is a valid tweet if
1. it has user\[id\} belonging to the list of listed ids and has embeded url
2. **if has a retweet that satisfies criteria 1 and the source tweet is published today**

In [5]:
# tweets = set()
actually_added = 0
parsed = 0
max_limit = 100
follow_list = [807095,1652541,3108351,51241574]
follow_str = ",".join([str(x) for x in follow_list])

In [ ]:
start = time.time()
print("Following users ",follow_str)
for tweet in t.filter(follow=follow_str):
    parsed += 1
    if parsed%100 == 0:
        print("Parsed :",parsed,"\n Added : ",actually_added)
    # tweet for a user we are following
    if 'retweeted_status' not in tweet:
        if tweet['user']['id'] in follow_list \
        and len(tweet['entities']['urls']) != 0 \
        and (df['tweet_id'] != tweet['id']).all():
            print("From source", tweet['id'])
            actually_added += 1
            current = datetime.datetime.now(datetime.timezone.utc)
            df = df.append({'screen_name':tweet['user']['screen_name'], 
                  'tweet_id':tweet['id'],
                  'created_at' : tweet['created_at'],
                  'embeded_url' : tweet['entities']['urls'][0]['url'],
                  'day_0_retweet_count': tweet['retweet_count'],
                  'day_0_time': current}, ignore_index=True)

    else:
        if tweet['retweeted_status']['user']['id'] in follow_list \
        and 'extended_tweet' in tweet['retweeted_status'] \
        and 'entities' in tweet["retweeted_status"]["extended_tweet"] \
        and len(tweet["retweeted_status"]["extended_tweet"]["entities"]['urls']) != 0\
        and (df['tweet_id'] != tweet['retweeted_status']['id']).all():
            created_time = datetime.datetime.strptime(tweet["retweeted_status"]["created_at"],"%a %b %d %H:%M:%S %z %Y")
            current = datetime.datetime.now(datetime.timezone.utc)
            diff =  current - created_time
            print(diff)
            if diff.days == 0:
                print("From retweet", tweet['id'],"adding tweet : ",tweet["retweeted_status"]["id"])
                df = df.append({'screen_name':tweet["retweeted_status"]["user"]["screen_name"], 
                      'tweet_id':tweet["retweeted_status"]["id"],
                      'created_at' : tweet["retweeted_status"]["created_at"],
                      'embeded_url' : tweet["retweeted_status"]["extended_tweet"]["entities"]["urls"][0]["url"],
                      'day_0_retweet_count': tweet['retweeted_status']['retweet_count'],
                      'day_0_time': current}, ignore_index=True)
                actually_added += 1
#             print(tweet['created_at'])
#             print(tweet['id'])
#             print(tweet['user']['id'])
#             print(tweet["retweeted_status"]["id"])
#             print(tweet["retweeted_status"]["created_at"])
#             print(tweet["retweeted_status"]["user"]["screen_name"])
#             print(tweet["retweeted_status"]["extended_tweet"]["entities"]["urls"][0]["url"])
    if actually_added == max_limit:
        break
print("Parsed :",parsed,"\n Added : ",actually_added,"in ", time.time()-start)

Following users  807095,1652541,3108351,51241574
0:12:54.435578
From retweet 1270156882687254533 adding tweet :  1270153658513207296
3 days, 13:39:13.610304
10:33:06.526132
From retweet 1270156941751455744 adding tweet :  1269997639992573954
0:03:14.239241
From retweet 1270156944825868290 adding tweet :  1270156154421690368
1 day, 23:53:12.104558
1:33:24.396740
From retweet 1270156995824431105 adding tweet :  1270133513891840002
4:18:27.888459
From retweet 1270157002002640897 adding tweet :  1270091981738274818
0:08:33.807928
From retweet 1270157026904010753 adding tweet :  1270154896105328640
0:15:52.661265
From retweet 1270157047229808645 adding tweet :  1270153075555172357
4:13:56.862870
From retweet 1270157052313247744 adding tweet :  1270093170479677441
1:53:34.507104
From retweet 1270157054976507904 adding tweet :  1270128497743343628
9:13:36.787590
From retweet 1270157064573202433 adding tweet :  1270017767186534400
13:04:41.626476
From retweet 1270157068054360064 adding tweet :

|Count|Time|
| ------------- |:-------------:| 
|10|123.57|

In [ ]:
df.head(max_limit)

In [ ]:
def get_retweet_count(twarc_api, id):
    for tweet in twarc_api.hydrate([id]):
        return tweet['retweet_count']
print(get_retweet_count(t, 1270044909886672896))

In [ ]:
for tweet in t.hydrate([1270148193792204802 ]):
    print(tweet['retweeted_status']['user']['id'] in follow_list)

for tweet in t.hydrate([1270044909886672896 ]):
    print(tweet['user']['id'] in follow_list)
#     print(tweet)
#     print(tweet['user']['screen_name'])
#     print(tweet['id'])
#     print(tweet['created_at'])
#     print(tweet['retweeted_status']['id'])
#     print(tweet['entities']['urls'][0]['url'])
#     print(tweet['retweeted_status'])
#     print(tweet['favorite_count'])
#     print("retweeted_status count", tweet['retweeted_status']['retweet_count'])
#     print(datetime.datetime.now())
#     df = df.append({'screen_name':tweet['user']['screen_name'], 
#               'tweet_id':tweet['id'],
#               'created_at' : tweet['created_at'],
#               'embeded_url' : tweet['entities']['urls'][0]['url'],
#               'day_0_retweet_count': tweet['retweet_count'],
#               'day_0_time':datetime.datetime.now()}, ignore_index=True)
# created_time = datetime.datetime.strptime(tweet['created_at'],"%a %b %d %H:%M:%S %z %Y")
# diff =  datetime.datetime.now(datetime.timezone.utc) - created_time
# abs(diff.days), datetime.datetime.now(datetime.timezone.utc), created_time

In [ ]:
# export_dataset(df)

In [ ]:
if (df['tweet_id'] == tweet['id']).any():
    print("yes")

In [ ]:
df.head()

In [ ]:
(df['tweet_id'] == tweet['id']).any()